In [ ]:
import numpy as np
import pandas as pd
import jieba
import os
import re

In [ ]:
os.chdir('C:/Users/64672/Desktop')

预处理

In [ ]:
data2 = pd.read_excel('地点或人物/伊景园.xlsx')

In [ ]:
data2['留言详情'] = data2['留言详情'].apply(lambda x : re.sub('\n','',str(x)))
data2['留言详情'] = data2['留言详情'].apply(lambda x : re.sub('\t','',str(x)))
data2['留言详情'] = data2['留言详情'].apply(lambda x : re.sub('\r','',str(x)))
data2['留言详情'] = data2['留言详情'].apply(lambda x : re.sub('\s','',str(x)))
data2['留言详情'] = data2['留言详情'].apply(lambda x : re.sub('http[A-Za-z:/.]+','',str(x)))

分词 （留言主题）

In [ ]:
with open('Python/stopwords.txt','r',encoding='gbk') as f:
    stop=f.read()
stop=stop.split()

In [ ]:
data2['topic']=data2['留言主题'].apply(jieba.lcut)
data2_cut=data2['topic'].apply(lambda x:[i for i in x if i not in stop])
data2_cut=data2_cut*2  #给留言主题比较大的权重

关键词

In [ ]:
import jieba.analyse
key=[]
for i in data2['留言详情']:
    keywords=jieba.analyse.extract_tags(i,topK=20)
    key.append(keywords)
data2['key']=key

In [ ]:
data2['key']=data2['key'].apply(lambda x:[i for i in x if i not in stop])

合并关键词

In [ ]:
for i in range(len(data2['key'])):
    data2['key'][i].extend(data2_cut[i])

计算相似度

In [ ]:
all_doc_list=data2['key'].tolist()

In [ ]:
from gensim import corpora,models,similarities
dictionary = corpora.Dictionary(all_doc_list)
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]
tfidf = models.TfidfModel(corpus)

In [ ]:
count=[]
id=[]
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=len(dictionary.keys()))
for i in range(len(data2['留言主题'])):
    sim = index[tfidf[corpus[i]]]
    a=sorted(enumerate(sim), key=lambda item: -item[1])
    a=pd.DataFrame(a)
    count.append((a[1]>0.1).sum())
    id.append(a.loc[a[1]>0.1,0].tolist())
data2['count']=count
data2['id']=id

In [ ]:
new_id=[]
for i in range(len(data2)):
    a=set(data2.iloc[i,10])
    for j in range(len(data2)):
        b=set(data2.iloc[j,10])
        if len(a&b) > 2:
            a = a|b
    new_id.append(a)

In [ ]:
def tostr(x):
    s=''
    for i in x:
        i=str(i)
        s=s+i
    return s

In [ ]:
data2['new_id'] = new_id 
data2['new_id'] = data2['new_id'].apply(lambda x: sorted(x))
data2['new_id2'] = data2['new_id'].apply(lambda x: tostr(x))

In [ ]:
data2_new=data2.drop_duplicates(subset='new_id2')

In [ ]:
data2_hot=data2.sort_values(by='new_id')
data2_hot.drop(data2_hot.columns[[7,8,9,10,11,12]],axis=1,inplace=True)

In [ ]:
data2=data2.sort_values(by='new_id')

In [ ]:
hot_id=[1]
j=1
for i in range(1,len(data2_hot)):
    if data2.iloc[i,12]==data2.iloc[i-1,12]:
        hot_id.append(j)
    else:
        j=j+1
        hot_id.append(j)
data2_hot['hot_id'] = hot_id

In [ ]:
data2_hot

In [ ]:
data2_hot.to_excel('伊景园.xlsx',index=False)